In [ ]:
# import most usefull libs
import pandas as pd
import numpy as np


#read data for model

wine_path=("../input/red-wine-quality-cortez-et-al-2009/winequality-red.csv")
dataset=pd.read_csv(wine_path)
dataset['quality_label'] = dataset.quality.apply(lambda q: 'low' if q <= 5 else 'medium' if q <= 7 else 'high')

dataset=pd.get_dummies(dataset)
dataset.head ()
#print(type(dataset))

In [ ]:
#check for missing values in the  columns
cols_missing=[col for col in dataset.columns
             if dataset[col].isnull().any()]
if cols_missing == []:
    print("Keine Missing Values")
    
#schauen ob der Datensatz kategorische Daten hat

list_cols_objects= (dataset.dtypes=="object")
object_cols= list(list_cols_objects[list_cols_objects].index)

if object_cols == []:
    print("Keine kategorischen Werte")
else :
    print("Kategorische Werte gefunden")


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.model_selection import train_test_split

#Werte in X und y splitten, in feature und target und preprocessor vorbereiten, dabei brauchen wir keinen OneHotEncoder oder SampleImputer

X= dataset.copy(deep=True)
colomns_for_x=dataset.columns [0:11]
X= dataset[colomns_for_x]
columns_for_y= dataset.columns[12:]
y= dataset[columns_for_y]


X_train, X_valid, y_train, y_valid = train_test_split(X,y, test_size=0.1)

sc= StandardScaler()
X_train = sc.fit_transform(X_train) #hier am besten den Durschnitt von allem oder nur von X_train
X_valid= sc.transform(X_valid)
input_shape = [X.shape[1]]
print("Input shape: {}".format(input_shape))

In [ ]:
"""#preprocessor auf X anwenden um die Werte zu standartisieren --> Vermutung auf bessere Performance
preprocessor= make_column_transformer(
                                    (StandardScaler(),
                                    make_column_selector(dtype_include=np.number)),
                                   )
X = preprocessor.fit_transform(X)
print("fertig")
input_shape = [X.shape[1]]
print("Input shape: {}".format(input_shape))
pd.DataFrame(X[:10,:]).head()""" #aus Exercise: Stochastic Gradient Descent könnte man auch nehmen

In [ ]:
X.head()

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

classification_model=keras.Sequential([
    layers.Dense(100, activation="relu", input_shape=input_shape), #100
    layers.Dense(200, activation="relu"), #200
    layers.Dense(64,activation="relu"),#64
    layers.Dense(3, activation='softmax') #Notiz: softmax oder sigmoid hat das Model extrem verbessern --> warum recherchieren!!!
])
print("finished")

In [ ]:

#model compilen und loss function (Ziel was trainiert werden sol z.B. Regression oder Klassifikation und Optimizer definiernén)
classification_model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics="accuracy",
    
)
print("finished")

In [ ]:
history = classification_model.fit(
            X, y,
            batch_size=60,
            epochs=10,
            
)

# Check your answer

In [ ]:
import pandas as pd

history_df = pd.DataFrame(history.history)
# Start the plot at epoch 5. You can change this to get a different view.
history_df.loc[5:, ['loss']].plot();

In [ ]:
y_pred= classification_model.predict(X_valid)
print(y_pred[0:10])

In [ ]:
y_pred_rounded = np.argmax(y_pred,axis=1)[:] #Returns the indices of the maximum values along an axis.
y_valid.head()

In [ ]:

y_valid_keine_Indices=y_valid.iloc[:,:].values #Indizes und labels aus dem altes df löschen

y_valid = np.argmax(y_valid_keine_Indices,axis=1)[:] #Returns the indices of the maximum values along an axis.
print(y_valid)

In [ ]:
for i in range(20):
    print("Predicted %d ---> Expected %d"%(y_pred_rounded[i],y_valid[i]))

In [ ]:
print(y_valid)

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


In [ ]:
import shap
shap.initjs()

# rather than use the whole training set to estimate expected values, we summarize with
# a set of weighted kmeans, each weighted by the number of points they represent.
X_train_summary = shap.kmeans(X_train, 10)

explainer = shap.KernelExplainer(classification_model.predict, X_train_summary)
shap_values = explainer.shap_values(X_valid)
shap.summary_plot(shap_values, X_valid)


print("DONE")